# BlackJack game:
#Objective: 

The objective is to make the sum of your card values as close to 21, without going over. 
If we make 21 exactly, we have blackjack, which can't be beat. If we go over 21, we 'bust' and we lose the round.

#The rules of blackjack are as follows:

1.It is a single player game.
2.The game has no gambling system.
3.Here we use only a single deck of cards.
4.The player and the dealer receive two cards from a shuffled deck. 
5.After the first two cards are dealt the player is asked if they would like another card (called 'hitting'), 
  or if they are happy with the cards they have already (called 'staying'). 
  The player is allowed to stop hitting at any point.
6.The number cards (2 through 10) are worth the number displayed, face cards are worth 10, and an Ace can be worth either 1 or     11.
7.Once our hand is finished, the dealer tries to do the same. The dealer must keep hitting until they get to 17. If they get       above 17 without busting, they can stay.

#Results:
1)If the player has blackjack, they win, unless the dealer also has blackjack, in which case the game is a tie.
2)If the dealer busts and the player doesn't, the player wins.
3)If the player busts, the dealer wins.
4)If the player and the dealer both don't bust, whoever is closest to 21 wins

References:
https://brilliant.org/wiki/programming-blackjack/
    
https://www.youtube.com/watch?v=qqp11XNMNRg
    
https://www.youtube.com/watch?v=yJz2at4Hco4


In [ ]:
from random import shuffle
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.neural_network import MLPClassifier

In [ ]:
#Creating a pandas data farme to store the output of each round.
#We store dealer count, player count ,hit or stand decision  and outcome.

columns=['Dealer_count','Player_count','Hit','Stand','Outcome']
df = pd.DataFrame(columns=columns)
print(df)

#use it store the number of previous increments 
previousinc=0;
#check if the player is bust or not
isbust=0;

# Listing cards and suits
a = [2,3,4,5,6,7,8,9,10,'JACK', 'QUEEN', 'KING', 'ACE']
b = ['SPADE','HEART ','DIAMOND','CLUB']

#getting a deck of cards as a list.
def get_new_deck(): 
    return [[rank, suit] for rank in a for suit in b]

#get cards 
cards=get_new_deck()

#return values for each card
def cardtotal(card):
    tmp = card[0]
    if tmp in a[0:8]:
        return int(tmp)
    elif tmp is 'ACE':
        return 11
    elif tmp is 'JACK' or 'QUEEN' or 'KING':
        return 10

#return the total of cards in hand
def handtotal(hand):
    tmp_value= 0
    aces=0

    #sum of cards in hand
    for x in hand:
        tmp_value = tmp_value + cardtotal(x)



    # Count the number of Aces in the hand.
    for y in hand:
        if y[0] is 'ACE':
            aces=aces + 1



    # Aces can count for 1, or 11.
    while aces > 0:
        if tmp_value > 21:
            tmp_value -= 10
            aces -= 1
        else:
            break

    # Return an integer representing the value of the hand. If 
    # the hand is bust, return 100.
    if tmp_value < 21:
        return tmp_value
    elif tmp_value == 21:
        return  21
    else:
        return tmp_value




#Loop for putting new predicted values into the model
for iteration in range(60):

    #If its the first iteration we provide data to train the model else we use the data generatedby the program as input
    if iteration == 0:
        data = pd.read_csv("C:\\Users\\Desktop\\RUTU\\DSP\\Project\\Final\\datainput.csv") 
    else:
        data = df
        
    
    #Training the Neural net which will be used to predict if the player should hit or stand.
    select_cols = ["Dealer_count","Player_count"]
    x = data[select_cols] #appending the columns to the dataset
    y= data.Hit #Defining our Y variable which is whether to hit or stand
    Neural_MLP = MLPClassifier(hidden_layer_sizes=(13,23,33),max_iter=2500,learning_rate_init=0.001,momentum=0.01,solver='adam')       
    Neural_MLP.fit(x,y)#Fitting the model with the classifier
    
    
    #Predicting values using the trained model.
    def Neural(x1):
        y1 = Neural_MLP.predict(x1)
        return y1

    
    #Run the game program and use the neural net to predict hit or stand value
    for item in range(50):
        
        #shuffling the deck of cards
        shuffle(cards)

       
        #check if player is in game or not
        still_playing = True
        increment=0
        round_result=0

        # issue the player and dealer their first two cards
        player_cards = [cards.pop(), cards.pop()]
        dealer_cards = [cards.pop(), cards.pop()]
        
        # As long as player is in the game ask them if they want to stay in the game (hit) or stand with their cards.
        while still_playing:

            # Display the player's current hand, as well as its value.
            current_score_str = '''\nYour current total is %s with cards in hand %s\n'''
            print (current_score_str % (handtotal(player_cards), player_cards))

            # If the player's hand is bust, don't ask them for a decision.
            if handtotal(player_cards) > 21:
                print ("Sorry your hand is bust!!!")
                break

            #if player is still playing as the model for hit or stand decision
            if still_playing :
                x1 = pd.DataFrame({'Dealer_count': handtotal(dealer_cards),
                            'Player_count':handtotal(player_cards)
                                 }, index=[0])
                response = Neural(x1)#get response from the model
                
                # If the player asks to be hit, take the first card from the top of
                # deck and add it to their hand. If they ask to stand, change
                # player_in to false, and move on to the dealer's hand.
                
                #if the response is hit
                if response:
                    still_playing = True
                    #Make a data frame of all values and append it to the orignal data frame
                    df1 = pd.DataFrame({'Dealer_count': handtotal(dealer_cards),
                            'Player_count':handtotal(player_cards),
                            'Hit':1,
                            'Stand':0, 
                            'Outcome':0}, index = [increment+item])
                    df=df.append(df1, ignore_index=True)
                    
                    #get new player card and append the new card to current list of cards.
                    new_player_card = cards.pop()
                    player_cards.append(new_player_card)
                    
                    #maintain value to update the outcome(win/loss) at the end of the game
                    increment+=1
                    print ('Your card is %s' % new_player_card)

                #if the response is stand
                else:
                    still_playing = False
                    
                    #Make a data frame of all values and append it to the orignal data frame
                    df1 = pd.DataFrame({'Dealer_count': handtotal(dealer_cards),
                            'Player_count':handtotal(player_cards),
                            'Hit':0,
                            'Stand':1, 
                            'Outcome':0}, index = [increment+item])
                    df=df.append(df1, ignore_index=True)
                    #maintain value to update the outcome(win/loss) at the end of the game
                    increment+=1
                    

        #If player hand goes above 21 dealer wins else if player stands dealer draws cards
        if handtotal(player_cards) <= 21:
            dealer_hand_string = '''\nDealer is at %s\nwith the hand %s\n'''
            print(dealer_hand_string % (str(handtotal(dealer_cards)), dealer_cards))
        else: 
            print ("Dealer wins.")
            round_result=0
            #set when player goes bust
            isbust=1



        #Dealer draws cards till they reach 17 and player is not bust
        while handtotal(dealer_cards)< 17 and handtotal(player_cards) <= 21:
            new_dealer_card = cards.pop()
            dealer_cards.append(new_dealer_card)
            print('Dealer draws %s' % new_dealer_card)




        #check final results
        if handtotal(player_cards) < 100 and handtotal(dealer_cards) > 21:
            print ('You beat the dealer!')
            round_result=1
        elif (handtotal(player_cards) > handtotal(dealer_cards)) and isbust == 0:
            print('You beat the dealer!')
            round_result=1
        elif handtotal(player_cards) == handtotal(dealer_cards):
            print ('You tied the dealer, nobody wins.')
            round_result=2
        elif handtotal(player_cards) < handtotal(dealer_cards):
            print ("Dealer wins!")
            round_result=0

        #For current round update win or loss
        for i in range(increment):
             df.loc[i+previousinc,'Outcome'] = round_result
        previousinc = previousinc + increment
        
        #reset the variable 
        isbust=0;
            

In [ ]:
df.to_csv('data.csv', encoding='utf-8', index=False)




